#TODO → Configure download folder

In [1]:
import helium as hell
from pathlib import Path
import time
import glob
import os
import sys
import getpass
#from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys

In [2]:
downloads='Downloads'
path='{}/{}'.format(os.getenv("HOME"),downloads)

In [3]:
url="https://login.webofknowledge.com"
url_login="https://login.ezproxy.unal.edu.co/login?url=http://www.webofknowledge.com"
#browser=hell.start_firefox(url_login)
browser=hell.start_chrome(url_login)

In [4]:
user=input('user:') #or email

user: anmsotohe


In [5]:
passwd=getpass.getpass('password:')

password: ·············


Configuration

In [6]:
country='Argentina'
UPDATE=False
UPDATE_from='2020'
SciELO=True

In [7]:
i=1
into_user=['Email address','usuario']
password=["Password","contraseña"]
tag_name=['Sign In','Ingresar']
extra_url=["",".ezproxy.unal.edu.co"]

In [8]:
hell.write(user,into=into_user[i])

In [9]:
hell.write(passwd,into=password[i])

<!-- 
email='restrepo@udea.edu.co'
passwd='666Opoqdh1!888'
-->

In [10]:
hell.click(tag_name[i])

In [11]:
try:
    browser.find_element_by_link_text('continue and establish a new session').click()
except:
    pass

In [38]:
#TODO → Select Scielo
if SciELO:
    br=browser.find_elements_by_class_name("select2-selection__rendered")    
    hell.click(br[0].text)
    hell.click('SciELO Citation Index')

In [39]:
hell.click('Advanced Search')

In [40]:
#time.sleep(10)
#qq=browser.find_element_by_name("currUrl")
#sid=qq.get_attribute('value').split('SID=')
#sid[-1]
#urls="https://apps.webofknowledge.com{}/WOS_AdvancedSearch_input.do?SID={}&product=WOS&search_mode=AdvancedSearch".format(extra_url[i],sid[-1])
#browser.get(urls)

In [41]:
browser.find_element_by_name("value(input1)").send_keys(f"CU={country}")

In [42]:
CONTINUE_UPDATE=False
br=browser.find_elements_by_class_name("select2-selection__rendered")
if UPDATE and br[1].text.find('All years')>-1:
    CONTINUE_UPDATE=True
    br[1].click()
if CONTINUE_UPDATE:
    time.sleep(1)    
    hell.click("Custom year range")
    time.sleep(1)
    kk=browser.find_elements_by_class_name("select2-selection__rendered")
    kk[2].click()
    kk=browser.find_element_by_class_name("select2-search__field")
    kk.send_keys(UPDATE_from)
    kk.send_keys(Keys.ENTER)

In [43]:
hell.click('Search')

In [44]:
urlr=browser.find_element_by_id('hitCount').get_attribute('href')

In [45]:
browser.get(urlr)

In [46]:
try:
    results=eval(browser.find_element_by_id('hitCount.top'
                               ).text.replace(',',''))
except:
    results=0
if not results:
    sys.exit()

In [47]:
wosmax=100000
results

37152

In [48]:
if results>wosmax:
    hell.click("more options / values...")
    time.sleep(5)

In [49]:
if results>wosmax:
    ob={}
    for j in range(1,100):
        try:
            br=browser.find_element_by_id(f'PY_{j}')
        except:
            break
        if br.get_attribute('value').find('PublicationYear_')>-1:
            year=br.get_attribute('value').split('_')[-1]
            if isinstance(year,str):
                ob[eval(year)]=br    
        else:
            break


    ct={}
    for rI in browser.find_elements_by_class_name('refineItem'):
        try:
            rIl=rI.text.split()
            ct[eval(rIl[0])]=eval(rI.text.split()[-1].replace(',',''))
        except:
            pass

    years=sorted(ct.keys())
    years.reverse()

    results=0
    for y in years:
        c=ct[y]
        if results+c<wosmax:
            results=results+c
            ob[y].click()
            time.sleep(1)            
        else:
            break
    time.sleep(1)
    bts=browser.find_element_by_class_name("ra_button_row")
    bts.find_element_by_tag_name("button").click()


In [50]:
#results=sum([ ct[yy] for yy in [y for y in years if y<2015 and y>2002  ] ] )
#results=16649

In [51]:
wostep=500
lstq=results%wostep
results=results-lstq

In [52]:
steps=list(range(1,results+wostep,wostep))

In [53]:
n=0
try:
    ll=[ n.split('(')[-1].split(')')[0] for n in sorted(glob.glob("{}/savedrecs(*.txt".format(path))) ]
    ll=sorted([eval(l) for l in ll])[::-1]
    n=ll[0]
except:
    pass

In [54]:
n=-1

In [55]:
steps=steps[n+1:]
steps[0]

35501

In [56]:
nstep=499
for step in steps:
    markFrom=step
    if step==steps[-2]:
        nstep=lstq-1
    martkTo=step+nstep
    print(markFrom,martkTo)
    break

35501 36000


In [57]:
slp=0.5
nextlooptime=10
markFrom=1
martTo=500
nstep=499
for step in steps:
    markFrom=step
    if step==steps[-2]:
        nstep=lstq-1
    martkTo=step+nstep
    print(markFrom,martkTo,n)
    
    #if step>1500:
    #    break
#if True:
    try:
        kk=browser.find_element_by_name('export').click()
        kk=browser.find_element_by_name(
            'Export to Other File Formats').click()
    except:
        kk=browser.find_elements_by_tag_name("button")
        i=[w.text for w in kk].index('Export to Other File Formats')
        kk[i].click()

    time.sleep(slp)
    kk=browser.find_element_by_id('numberOfRecordsRange').click()

    time.sleep(slp)
    kk=browser.find_element_by_id('markFrom').clear()
    kk=browser.find_element_by_id('markFrom').send_keys("{}".format(markFrom))

    time.sleep(slp)
    kk=browser.find_element_by_id('markTo').clear()
    kk=browser.find_element_by_id('markTo').send_keys("{}".format(martkTo))

    time.sleep(slp)
    #Take the id of the <span id='select2-bib_fields-container'
    kk=browser.find_element_by_id('select2-bib_fields-container')
    #make click to activate the hidden `<li` options!
    kk.click()

    #Get the dinamically generated new `<li` options
    i=[w.text for w in browser.find_elements_by_tag_name('li')].index('Full Record and Cited References')

    time.sleep(slp)
    browser.find_elements_by_tag_name('li')[i].click()

    #The same for the next dinamycal menu

    time.sleep(slp)
    kk=browser.find_element_by_id('select2-saveOptions-container').click()

    i=[w.text for w in browser.find_elements_by_tag_name('li')].index('Plain Text')

    time.sleep(slp)
    browser.find_elements_by_tag_name('li')[i].click()

    time.sleep(slp)
    #Firefox download popup must be configured to save automatically the first time!!!!!
    browser.find_element_by_id("exportButton").click()

    #wtr=range(10)
    #for wait in wtr:
    #    if glob.glob("{}/savedrecs({}).txt".format(path,n+1)):
    #        print('va')
    if True:
            n=n+1
    #        break
    #    else:
    #        print('.',end='')
    #        time.sleep(nextlooptime)
    print('')
    #TODO: break for n=0
    #if wait==wtr[-1] and n>0:
    #    print('WARNING: Download failed')

    time.sleep(slp)
    #kk=browser.find_element_by_class_name("quickoutput-overlay-buttonset")
    #kk.find_element_by_tag_name('a').click()
    time.sleep(nextlooptime)

#TODO:
# * Make a logout

35501 36000 70

36001 36500 71

36501 36652 72

37001 37152 73



In [58]:
browser.close()